In [38]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/data/xzhao/experiments/roman-pretrain/exps/exp1-multi-ja/hf_models/iter_0000010")

/home/xzhao/.pyenv/versions/poetry-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
len(list(tokenizer.get_vocab().values()))

99574

In [54]:
ids = tokenizer.encode("次の文章に続きの文を加えてください。")
tokenizer.decode(ids)

'<s> 次の文章に続きの文を加えてください。'

In [52]:
from utils import load_json


item = load_json("/data/xzhao/experiments/med-eval/dataset/losses/results/random_2k/llm-jp-3-13b-exp1-multi-ja/iter_0000010_ja_full_original.json")

JSONDecodeError: Extra data: line 1139 column 1 (char 15389)

In [9]:
tokenizer.unk_token, tokenizer.unk_token_id

('<unk>', 0)

In [47]:
tokenizer("This is a test sentence.", return_tensors="pt")['input_ids'][0]

tensor([    1,   863,   367,   283,  1381, 12279, 28728])

In [77]:
from utils import load_jsonl_iteratively


filepath = "/data/xzhao/experiments/med-eval/dataset/losses/results/random_2k/ja_full_retoken-4@1/llm-jp-3-13b-exp1-multi-ja/iter_0000010.jsonl"
for item in load_jsonl_iteratively(filepath, request_num=3):

    print('-------before------')
    print(item['metadata']['org_text'])
    print(item['metadata']['token_labels'])
    print('-------after------')
    print(item['text'])
    print()

-------before------
次の文章に続きの文を加えてください。
我々は, 細菌感染によってウサギに慢性副鼻腔炎を惹起することにより, 副鼻腔及び鼻腔にポリープが形成されるのを認めた。...
これらは組織学的特徴から,「肉芽性ポリープ」および「浮腫性ポリープ」の二つの型に分けられた。 「肉芽性ポリープ」は上皮を完全に失った部で, 線維芽細胞の増殖と血管新生が起こって肉芽が洞内に突出することによって形成され, それを被うように未分化な上皮細胞が増殖移動していくように観察された。 「浮腫性ポリープ」は, 上皮細胞が脱落したり, 成熟した形態を失って背の低い未分化な形態を示したり, 扁平な上皮細胞が重層して観察されたりする部で, 形成されていた。 本研究の結果は, 上皮細胞も鼻茸形成の初期において, なんらかの役割を果たしていることを示唆する。
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 46, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 110, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145

In [62]:
import random

def shuffle_list_by_steps(lst, steps, window_size, seed=None):
    """
    Shuffle a list by a given number of steps.
    """
    if steps <= 0 or steps > len(lst):
        raise ValueError("Steps must be greater than 0 and less than the length of the list. But got steps: {}, length: {}".format(steps, len(lst)))
    
    
    if seed is not None:
        random.seed(seed)

    
    orders = list(range(len(lst)))
    modified = 0
    while modified < steps:
        src = random.randint(0, len(lst) - 1)
        window = [i for i in range(src - window_size, src + window_size) if i != src and 0 <= i < len(lst)]
        tgt = random.choice(window)
        # print("src: {}, tgt: {}; orders[src]: {}, orders[tgt]: {}".format(src, tgt, orders[src], orders[tgt]))

        if steps - modified == 1 and orders[src] == src and orders[tgt] == tgt:
            continue
            
        if orders[src] == src:
            modified += 2 if orders[tgt] == tgt else 1
            orders[src], orders[tgt] = orders[tgt], orders[src]
        else:
            if orders[tgt] == tgt:
                modified += 1
                orders[src], orders[tgt] = orders[tgt], orders[src]
            
    assert sum([i != j for i, j in zip(orders, range(len(lst)))]) == steps, \
        "Shuffle steps do not match. Modification: {}, Expected: {}, Actual: {}".format(
        modified, steps, sum([i != j for i, j in zip(orders, range(len(lst)))])
    )
    print("=========")
    print("before: ", lst)
    print(orders)
    print("after: ", [lst[i] for i in orders])
    return [lst[i] for i in orders]

In [63]:
shuffle_list_by_steps(list(range(4)), 4, window_size=1)

before:  [0, 1, 2, 3]
[2, 0, 3, 1]
after:  [2, 0, 3, 1]


[2, 0, 3, 1]

In [6]:
tokenizer("this is a test")['input_ids']  

[1, 471, 367, 283, 1381]

In [1]:
import os
import torch
import argparse
from tqdm import tqdm

from utils import load_jsonl, dump_json
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader
import torch.nn.functional as F

# args = parser.parse_args()

args = {
    "model_name": "llm-jp-3-13b-exp1-en_jstage",
    "model_path": "/model/projects/cpt-scidoc/roman-pretrain/exp1-en_jstage/iter_0000001",
    "iteration": "iter_0000001",
    "batch_size": 1
}

dump_root = "/home/xzhao/workspace/med-eval/dataset/losses/results"
dump_root = os.path.join(dump_root, args['model_name'])
os.makedirs(dump_root, exist_ok=True)
dump_file = os.path.join(dump_root, f"{args['iteration']}.json")

assert os.path.exists(args['model_path']), args['model_path']


print(f"Dump file: {dump_file}")
print(f"Model name: {args['model_name']}")
print(f"Iteration: {args['iteration']}")
print(f"Model path: {args['model_path']}")
print(f"Batch size: {args['batch_size']}") 

# if os.path.exists(dump_file):
#     print(f"{dump_file} already exists, skip")
#     exit(0)

tokenizer = AutoTokenizer.from_pretrained(args['model_path'])
model = AutoModelForCausalLM.from_pretrained(args['model_path'], device_map="auto")
model.eval()

train_samples_root = "/home/xzhao/workspace/med-eval/dataset/losses/train_samples"
filename_dict = {
    # "en": os.path.join(train_samples_root, "en_10k.jsonl"),
    # "ja": os.path.join(train_samples_root, "ja_10k.jsonl"),
    "en_jstage": os.path.join(train_samples_root, "en_jstage_10k.jsonl"),
}

/home/xzhao/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dump file: /home/xzhao/workspace/med-eval/dataset/losses/results/llm-jp-3-13b-exp1-en_jstage/iter_0000000.json
Model name: llm-jp-3-13b-exp1-en_jstage
Iteration: iter_0000000
Model path: /model/projects/cpt-scidoc/roman-pretrain/exp1-en_jstage/iter_0000001
Batch size: 1


Loading checkpoint shards: 100%|██████████| 6/6 [03:27<00:00, 34.54s/it]


In [2]:
losses = {}
for train_type, path in filename_dict.items():
    sequences = [item['text'] for item in load_jsonl(path)]
    break

In [4]:
inputs = tokenizer(sequences[0], return_tensors="pt")
input_ids = inputs["input_ids"].to(model.device)
with torch.no_grad():
    outputs = model(input_ids=input_ids)
    # Shift input_ids to get labels for next-token prediction
    shift_logits = outputs.logits[:, :-1, :].contiguous()
    shift_labels = input_ids[:, 1:].contiguous()

    # Compute per-token loss (no reduction)
    per_token_loss = F.cross_entropy(
        shift_logits.view(-1, shift_logits.size(-1)),
        shift_labels.view(-1),
        reduction='none'
    ).view(shift_labels.size())


In [7]:
input_ids.shape, per_token_loss.shape

(torch.Size([1, 85]), torch.Size([1, 84]))

In [24]:
def caluate_loss_single_per_token(model, tokenizer, sequences):
    total_loss = 0
    seq2loss = {}
    for seq in tqdm(sequences):
        inputs = tokenizer(seq, return_tensors="pt")
        input_ids = inputs["input_ids"].to(model.device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids)
            shift_logits = outputs.logits[:, :-1, :].contiguous()
            shift_labels = input_ids[:, 1:].contiguous()

            per_token_loss = F.cross_entropy(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1),
                reduction='none'
            ).view(shift_labels.size())
        
        token_losses = []
        for label, loss in zip(shift_labels[0].tolist(), per_token_loss[0].tolist()):
            token_losses.append((tokenizer.decode(label), label, loss))
        token_losses = tuple(token_losses)
        
        loss = per_token_loss[0].mean().item()
        seq2loss[seq] = {
            "loss": float(loss),
            "token_losses": token_losses,
        }
        total_loss += loss
    average_loss = total_loss / len(sequences)
    print(f"\nAverage loss: {average_loss:.4f}")
    return seq2loss, average_loss

In [25]:
seq2loss, average_loss = caluate_loss_single_per_token(model, tokenizer, sequences[0:10])

100%|██████████| 10/10 [00:03<00:00,  3.16it/s]


Average loss: 2.2395


In [26]:
seq2loss

{'Given the following question, provide an answer: How was moderate postoperative anemia diagnosed in patients after OPCAB?\nModerate postoperative anemia was diagnosed based on a hemoglobin concentration lower than 10 g/dl for three consecutive postoperative days. This criterion ensures that the diagnosis reflects a sustained rather than transient condition, providing a clear clinical marker.\n\n---': {'loss': 2.277425527572632,
  'token_losses': (('Given', 12636, 8.787433624267578),
   ('the', 291, 0.893963098526001),
   ('following', 2306, 4.7838454246521),
   ('question', 3005, 2.278109073638916),
   (',', 28730, 1.150255799293518),
   ('provide', 3093, 3.5563912391662598),
   ('an', 412, 1.245552897453308),
   ('answer', 4381, 0.22238853573799133),
   (':', 28752, 0.232383131980896),
   ('How', 1613, 2.3880748748779297),
   ('was', 419, 4.524470329284668),
   ('moderate', 22658, 12.618608474731445),
   ('post', 1715, 8.318605422973633),
   ('oper', 3024, 1.098888635635376),
   ('a

In [ ]:
with torch.no_grad():
    outputs = model(input_ids=input_ids, labels=input_ids)

2.2774252891540527

In [23]:
outputs.loss

tensor(2.2774, device='cuda:0')